In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [3]:
if not os.path.exists('./mlp_img_Dae'):
    os.mkdir('./mlp_img_Dae')

In [4]:
def to_img(x):
    x = x.view(x.size(0),1,28,28)
    #print (x)
    return x

In [5]:
num_epochs = 10
batch_size = 128
learning_rate = 0.001

In [6]:
def plot_sample_img(img, name):
    img=img.view(1,28,28)
    save_img(img, './sample_{}.png'.format(name))

In [7]:
def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor/max_tensor
    tensor = tensor*(max_value - min_value) + min_value
    return torch.round(tensor)

In [8]:
img_transform =transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:min_max_normalization(tensor, 0,1))
    
])

In [9]:
dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle= True)

In [10]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder,self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28,256),
            nn.ReLU(True),
            nn.Linear(256,64),
            nn.ReLU(True))
        
        self.decoder = nn.Sequential(
            nn.Linear(64,256),
            nn.ReLU(True),
            nn.Linear(256,28*28),
            nn.Sigmoid())
        # Define forward
    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [11]:
model = autoencoder().cuda()
model

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU(inplace)
  )
  (decoder): Sequential(
    (0): Linear(in_features=64, out_features=256, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=256, out_features=784, bias=True)
    (3): Sigmoid()
  )
)

In [12]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=0.0001)

In [13]:
# Training

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0),-1)
        img = Variable(img).cuda()
        
        # Forward
        output = model(img)
        loss = criterion(output, img)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # log
        print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.item()))
        if epoch%10==0:
            x = to_img(img.cpu().data)
            x_hat = to_img(output.cpu().data)
            save_image(x,'./mlp_img_dae/x_{}.png'.format(epoch))
            save_image(x,'./mlp_img_dae/x_hat{}.png'.format(epoch))
            
torch.save(model.state_dict(),'./sim_denoising_autoencoder.pth')
        

epoch [1/10], loss:0.2501
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
     

          [0.4996, 0.4830, 0.4980,  ..., 0.5034, 0.4944, 0.4818]]]])
epoch [1/10], loss:0.2446
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0.

          [0.4996, 0.4685, 0.4786,  ..., 0.4873, 0.4804, 0.4612]]]])
epoch [1/10], loss:0.2331
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0.

          [0.4852, 0.4440, 0.4622,  ..., 0.4666, 0.4700, 0.4373]]]])
epoch [1/10], loss:0.2113
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0.

          [0.4737, 0.4260, 0.4515,  ..., 0.4558, 0.4580, 0.4304]]]])
epoch [1/10], loss:0.1716
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0.

...

epoch [4/10], loss:0.0391
epoch [4/10], loss:0.0387
epoch [4/10], loss:0.0386
epoch [4/10], loss:0.0371
epoch [4/10], loss:0.0413
epoch [4/10], loss:0.0383
epoch [4/10], loss:0.0399
epoch [4/10], loss:0.0375
epoch [4/10], loss:0.0401
epoch [4/10], loss:0.0395
epoch [4/10], loss:0.0388
epoch [4/10], loss:0.0396
epoch [4/10], loss:0.0420
epoch [4/10], loss:0.0420
epoch [4/10], loss:0.0399
epoch [4/10], loss:0.0394
epoch [4/10], loss:0.0383
epoch [4/10], loss:0.0398
epoch [4/10], loss:0.0405
epoch [4/10], loss:0.0389
epoch [4/10], loss:0.0365
epoch [4/10], loss:0.0411
epoch [4/10], loss:0.0388
epoch [4/10], loss:0.0385
epoch [4/10], loss:0.0395
epoch [4/10], loss:0.0408
epoch [4/10], loss:0.0425
epoch [4/10], loss:0.0371
epoch [4/10], loss:0.0404
epoch [4/10], loss:0.0376
epoch [4/10], loss:0.0401
epoch [4/10], loss:0.0405
epoch [4/10], loss:0.0414
epoch [4/10], loss:0.0383
epoch [4/10], loss:0.0413
epoch [4/10], loss:0.0382
epoch [4/10], loss:0.0392
epoch [4/10], loss:0.0398
epoch [4/10]

epoch [5/10], loss:0.0372
epoch [5/10], loss:0.0381
epoch [5/10], loss:0.0378
epoch [5/10], loss:0.0374
epoch [5/10], loss:0.0378
epoch [5/10], loss:0.0359
epoch [5/10], loss:0.0400
epoch [5/10], loss:0.0383
epoch [5/10], loss:0.0370
epoch [5/10], loss:0.0377
epoch [5/10], loss:0.0394
epoch [5/10], loss:0.0387
epoch [5/10], loss:0.0382
epoch [5/10], loss:0.0382
epoch [5/10], loss:0.0387
epoch [5/10], loss:0.0364
epoch [5/10], loss:0.0357
epoch [5/10], loss:0.0363
epoch [5/10], loss:0.0378
epoch [5/10], loss:0.0390
epoch [5/10], loss:0.0372
epoch [5/10], loss:0.0374
epoch [5/10], loss:0.0420
epoch [5/10], loss:0.0382
epoch [5/10], loss:0.0396
epoch [5/10], loss:0.0383
epoch [5/10], loss:0.0383
epoch [5/10], loss:0.0379
epoch [5/10], loss:0.0360
epoch [5/10], loss:0.0382
epoch [5/10], loss:0.0371
epoch [5/10], loss:0.0367
epoch [5/10], loss:0.0373
epoch [5/10], loss:0.0378
epoch [5/10], loss:0.0371
epoch [5/10], loss:0.0379
epoch [5/10], loss:0.0396
epoch [5/10], loss:0.0367
epoch [5/10]

epoch [7/10], loss:0.0364
epoch [7/10], loss:0.0385
epoch [7/10], loss:0.0365
epoch [7/10], loss:0.0362
epoch [7/10], loss:0.0386
epoch [7/10], loss:0.0384
epoch [7/10], loss:0.0376
epoch [7/10], loss:0.0381
epoch [7/10], loss:0.0381
epoch [7/10], loss:0.0375
epoch [7/10], loss:0.0368
epoch [7/10], loss:0.0381
epoch [7/10], loss:0.0388
epoch [7/10], loss:0.0408
epoch [7/10], loss:0.0364
epoch [7/10], loss:0.0375
epoch [7/10], loss:0.0358
epoch [7/10], loss:0.0371
epoch [7/10], loss:0.0374
epoch [7/10], loss:0.0371
epoch [7/10], loss:0.0365
epoch [7/10], loss:0.0376
epoch [7/10], loss:0.0361
epoch [7/10], loss:0.0366
epoch [7/10], loss:0.0379
epoch [7/10], loss:0.0370
epoch [7/10], loss:0.0351
epoch [7/10], loss:0.0365
epoch [7/10], loss:0.0379
epoch [7/10], loss:0.0373
epoch [7/10], loss:0.0380
epoch [7/10], loss:0.0365
epoch [7/10], loss:0.0360
epoch [7/10], loss:0.0385
epoch [7/10], loss:0.0384
epoch [7/10], loss:0.0361
epoch [7/10], loss:0.0381
epoch [7/10], loss:0.0380
epoch [7/10]

epoch [8/10], loss:0.0363
epoch [8/10], loss:0.0354
epoch [8/10], loss:0.0351
epoch [8/10], loss:0.0349
epoch [8/10], loss:0.0359
epoch [8/10], loss:0.0366
epoch [8/10], loss:0.0363
epoch [8/10], loss:0.0383
epoch [8/10], loss:0.0361
epoch [8/10], loss:0.0358
epoch [8/10], loss:0.0372
epoch [8/10], loss:0.0382
epoch [8/10], loss:0.0372
epoch [8/10], loss:0.0363
epoch [8/10], loss:0.0364
epoch [8/10], loss:0.0373
epoch [8/10], loss:0.0363
epoch [8/10], loss:0.0369
epoch [8/10], loss:0.0355
epoch [8/10], loss:0.0362
epoch [8/10], loss:0.0366
epoch [8/10], loss:0.0374
epoch [8/10], loss:0.0380
epoch [8/10], loss:0.0370
epoch [8/10], loss:0.0365
epoch [8/10], loss:0.0376
epoch [8/10], loss:0.0362
epoch [8/10], loss:0.0356
epoch [8/10], loss:0.0358
epoch [8/10], loss:0.0365
epoch [8/10], loss:0.0347
epoch [8/10], loss:0.0361
epoch [8/10], loss:0.0379
epoch [8/10], loss:0.0362
epoch [8/10], loss:0.0368
epoch [8/10], loss:0.0354
epoch [8/10], loss:0.0396
epoch [8/10], loss:0.0362
epoch [8/10]

epoch [9/10], loss:0.0333
epoch [9/10], loss:0.0370
epoch [9/10], loss:0.0370
epoch [9/10], loss:0.0392
epoch [9/10], loss:0.0369
epoch [9/10], loss:0.0380
epoch [9/10], loss:0.0350
epoch [9/10], loss:0.0373
epoch [9/10], loss:0.0363
epoch [9/10], loss:0.0369
epoch [9/10], loss:0.0385
epoch [9/10], loss:0.0376
epoch [9/10], loss:0.0372
epoch [9/10], loss:0.0346
epoch [9/10], loss:0.0384
epoch [9/10], loss:0.0354
epoch [9/10], loss:0.0386
epoch [9/10], loss:0.0374
epoch [9/10], loss:0.0367
epoch [9/10], loss:0.0364
epoch [9/10], loss:0.0375
epoch [9/10], loss:0.0365
epoch [9/10], loss:0.0350
epoch [9/10], loss:0.0377
epoch [9/10], loss:0.0393
epoch [9/10], loss:0.0356
epoch [9/10], loss:0.0358
epoch [9/10], loss:0.0356
epoch [9/10], loss:0.0372
epoch [9/10], loss:0.0365
epoch [9/10], loss:0.0382
epoch [9/10], loss:0.0364
epoch [9/10], loss:0.0355
epoch [9/10], loss:0.0368
epoch [9/10], loss:0.0359
epoch [9/10], loss:0.0362
epoch [9/10], loss:0.0357
epoch [9/10], loss:0.0376
epoch [9/10]

In [1]:
os.getcwd()

NameError: name 'os' is not defined

RuntimeError: invalid argument 2: size '[1 x 28 x 28]' is invalid for input with 100352 elements at ..\aten\src\TH\THStorage.cpp:84